In [139]:
reload_lamb()

In [140]:
class TVStoreExpr(meta.Tuple):
    def __init__(self, arg, tv_layer, typ=None):
        arg = self.ensure_typed_expr(arg, typ=types.type_t)
        meta.Tuple.__init__(self, (arg, tv_layer), typ=type)

    def copy(self):
        return TVStoreExpr(self.args)

    def term(self):
        return False

    def tuple(self):
        return tuple(self.args)

    def __repr__(self):
        return "(" + ", ".join([repr(a) for a in self.args]) + ")"

    def latex_str(self, parens=True):
        inner = ", ".join([a.latex_str() for a in self.args])
        if parens:
            return ensuremath("(" + inner + ")")
        else:
            return ensuremath(inner)

In [141]:
%%lamb
||danced|| = L x_e : Dance(x)
||saw|| = L x_e : L y_e : (Saw(y,x), PhysExistence(x) & Sensory(y))
||the|| = L f_<e,t> : ((Iota x_e :f(x)), (Exists x_e : f(x)))
||cat|| = L x_e : Cat(x)
||Bill|| = Bill_e

INFO (meta): Coerced guessed type t for 'Dance_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'
INFO (meta): Coerced guessed type t for 'PhysExistence_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'Sensory_t' into <e,t>, to match argument 'y_e'
INFO (meta): Coerced guessed type t for 'Cat_t' into <e,t>, to match argument 'x_e'


$||\mathbf{\text{danced}}||^{}_{\langle{}e,t\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Dance}({x}_{e})$<br />
$||\mathbf{\text{saw}}||^{}_{\langle{}e,\langle{}e,(t, t)\rangle{}\rangle{}} \:=\: \lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: ({Saw}({y}_{e}, {x}_{e}), ({PhysExistence}({x}_{e}) \wedge{} {Sensory}({y}_{e})))$<br />
$||\mathbf{\text{the}}||^{}_{\langle{}\langle{}e,t\rangle{},(e, t)\rangle{}} \:=\: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: (\iota{} x_{e} \: . \: {f}_{\langle{}e,t\rangle{}}({x}_{e}), \exists{} x_{e} \: . \: {f}_{\langle{}e,t\rangle{}}({x}_{e}))$<br />
$||\mathbf{\text{cat}}||^{}_{\langle{}e,t\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Cat}({x}_{e})$<br />
$||\mathbf{\text{Bill}}||^{}_{e} \:=\: {Bill}_{e}$

In [142]:
tc = the * cat
tc

CompositionResult(results=[||[the cat]|| = ((ι x_e: Cat_<e,t>(x_e)), (Exists x_e: Cat_<e,t>(x_e)))], failures=[||[cat the]|| = Type mismatch: '||cat|| = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '||the|| = (λ f_<e,t>: ((ι x_e: f_<e,t>(x_e)), (Exists x_e: f_<e,t>(x_e))))'/<<e,t>,(e,t)> conflict (mode: Function Application), ||[the cat]|| = Type mismatch: '||the|| = (λ f_<e,t>: ((ι x_e: f_<e,t>(x_e)), (Exists x_e: f_<e,t>(x_e))))'/<<e,t>,(e,t)> and '||cat|| = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Modification), ||[the cat]|| = Type mismatch: '||the|| = (λ f_<e,t>: ((ι x_e: f_<e,t>(x_e)), (Exists x_e: f_<e,t>(x_e))))'/<<e,t>,(e,t)> and '||cat|| = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ||[cat the]|| = Type mismatch: '||cat|| = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '||the|| = (λ f_<e,t>: ((ι x_e: f_<e,t>(x_e)), (Exists x_e: f_<e,t>(x_e))))'/<<e,t>,(e,t)> conflict (mode: Predicate Abstraction)])

In [143]:
#fun = te("L f_<e,t> : L a_(e,t) : (f(a[0]), a[1])")
def project_fa(fun, arg):
    if not isinstance(fun.type, types.FunType):
        raise types.TypeMismatch(fun, arg, "Project")
    if not isinstance(arg.type, types.TupleType):
        raise types.TypeMismatch(fun, arg, "Project")
    if not fun.type.left == arg.type[0]: # TODO: use unify
        raise types.TypeMismatch(fun, arg, "Project")
    result = meta.Tuple((fun(arg.args[0]), arg.args[1]))
    return result

project_fa(danced.content, tc[0].content).reduce_all()

(Dance_<e,t>((ι x_e: Cat_<e,t>(x_e))), (Exists x_e: Cat_<e,t>(x_e)))

In [144]:
tc[0]

||[the cat]|| = ((ι x_e: Cat_<e,t>(x_e)), (Exists x_e: Cat_<e,t>(x_e)))

In [145]:
def projective_tuple_type(t):
    if isinstance(t, types.TupleType):
        if len(t) == 2:
            if t[1] == types.type_t:
                return True
    return False

def project_combine(rule):
    def fun(i1, i2, assignment=None):
        if projective_tuple_type(i1.type):
            i1_un = lang.TreeComposite(content=i1.content.args[0])
            if projective_tuple_type(i2.type):
                i2_un = lang.TreeComposite(content=i2.content.args[0])
                result = rule(i1_un, i2_un, assignment=assignment)
                project = il.content.args[1] & i2.content.args[1]
            else:
                result = rule(i1_un, i2, assignment=assignment)
                project = i1.content.args[1]
        else:
            if projective_tuple_type(i2.type):
                i2_un = lang.TreeComposite(content=i2.content.args[0])
                result = rule(i1, i2_un, assignment=assignment)
                project = i2.content.args[1]
            else:
                result = rule(i1, i2, assignment=assignment)
                project = None
        if projective_tuple_type(result.type):
            # merge projected meaning resulting from application of `rule` with projected meaning already present
            if project:
                project = project & result.content.index(1)
            else:
                project = result.content.index(1)
            result.content = result.content.index(0) # hope this works!
        if project:
            # discard intermediate Composite object probably created by application of rule
            full_result_te = meta.Tuple((result.content, project))
            full_result = lang.BinaryComposite(i1, i2, full_result_te)
        else:
            full_result = result
        return full_result
    return fun
    
project_fa = project_combine(lang.fa_fun)
            

In [146]:
project_fa(danced, tc[0])

||[danced [the cat]]|| = (Dance_<e,t>((ι x_e: Cat_<e,t>(x_e))), (Exists x_e: Cat_<e,t>(x_e)))

In [147]:
project_fa(saw, tc[0])

||[saw [the cat]]|| = ((λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: Cat_<e,t>(x_e))), (PhysExistence_<e,t>((ι x_e: Cat_<e,t>(x_e))) & Sensory_<e,t>(y_e)))), (Exists x_e: Cat_<e,t>(x_e)))

In [148]:
project_fa(project_fa(saw, tc[0]), Bill)

||[[saw [the cat]] Bill]|| = (Saw_<(e,e),t>(Bill_e, (ι x_e: Cat_<e,t>(x_e))), ((Exists x_e: Cat_<e,t>(x_e)) & (PhysExistence_<e,t>((ι x_e: Cat_<e,t>(x_e))) & Sensory_<e,t>(Bill_e))))

In [149]:
%%lamb
||everyone|| = L f_<e,t> : Forall x_e : f(x)

$||\mathbf{\text{everyone}}||^{}_{\langle{}\langle{}e,t\rangle{},t\rangle{}} \:=\: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: \forall{} x_{e} \: . \: {f}_{\langle{}e,t\rangle{}}({x}_{e})$

In [150]:
project_fa(everyone, project_fa(saw, tc[0]))

TypeMismatch: Type mismatch: '||everyone|| = (λ f_<e,t>: (Forall x_e: f_<e,t>(x_e)))'/<<e,t>,t> and '|||| = (λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: Cat_<e,t>(x_e))), (PhysExistence_<e,t>((ι x_e: Cat_<e,t>(x_e))) & Sensory_<e,t>(y_e))))'/<e,(t,t)> conflict (mode: Function Application)

In [151]:
def lift_to_projtype(t):
    if t.type.functional():
        comb = te("L f_%s : L a1_(%s,t) : (f(a1[0]), a1[1])" % (t.type, t.type.left))
    else:
        comb = te("L f_%s : (f, True_t)" % t.type)
    #return comb(t).reduce_all()
    return comb(t)

def lower_from_projtype(t):
    if t.type.functional():
        comb1 = te("L f_%s : L a1_%s : (f(a1))[0]" % (t.type, t.type.left[0]))
        comb2 = te("L f_%s : L a2_%s : (f(a2))[1]" % (t.type, t.type.left[1]))
        r1 = comb1(t).reduce_all()
        r2 = comb2(t).reduce_all()
        return (r1, r2)
    else:
        return t.index(0), t.index(1)

In [152]:
#te("L f_<e,t> : L a1_(e,t)  : (f(a1[0]), a1[1])")
te("L a_(e,t) : a[1]")

(λ a_(e,t): (a_(e,t)[1_n]))

In [153]:
lift_to_projtype(danced.content)

((λ f_<e,t>: (λ a1_(e,t): (f_<e,t>((a1_(e,t)[0_n])), (a1_(e,t)[1_n])))))((λ x_e: Dance_<e,t>(x_e)))

In [154]:
lower_from_projtype(the.content)

TypeError: 'FunType' object does not support indexing

In [155]:
x = lift_to_projtype(the.content)
x.reduce_all()

(λ a1_(<e,t>,t): (((ι x_e: (a1_(<e,t>,t)[0_n])(x_e)), (Exists x_e: (a1_(<e,t>,t)[0_n])(x_e))), (a1_(<e,t>,t)[1_n])))

In [189]:
def merge_proj(target, addition):
    if projective_tuple_type(target.type):
        return meta.Tuple((target.index(0), target.index(1) & addition))
    else:
        return meta.Tuple((target, addition))

def proj_apply(f, a, assignment=None):
    try:
        result = lang.fa_fun(f, a, assignment)
        return result
    except types.TypeMismatch as e:
        if projective_tuple_type(f.type):
            f0 = f.content.under_assignment(assignment).index(0)
            f1 = f.content.under_assignment(assignment).index(1)
            result = proj_apply(lang.TreeComposite(content=f0), a, assignment)
            new_content = merge_proj(result.content, f1)
            return lang.BinaryComposite(f, a, new_content)
        elif f.type.functional() and projective_tuple_type(a.type) and not projective_tuple_type(f.type.left):
            # need to check for functional here because a TM may be thrown on a non-functional value for f
            a0 = a.content.under_assignment(assignment).index(0)
            a1 = a.content.under_assignment(assignment).index(1)
            result = proj_apply(f, lang.TreeComposite(content=a0), assignment)
            new_content = merge_proj(result.content, a1)
            return lang.BinaryComposite(f, a, new_content)
        else:
            raise e

In [190]:
proj_apply(saw, proj_apply(the, cat)).reduce_all()

||[saw [the cat]]|| = ((λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: Cat_<e,t>(x_e))), (PhysExistence_<e,t>((ι x_e: Cat_<e,t>(x_e))) & Sensory_<e,t>(y_e)))), (Exists x_e: Cat_<e,t>(x_e)))

In [192]:
proj_apply(everyone, proj_apply(saw, proj_apply(the, cat))).reduce_all()

TypeMismatch: Type mismatch: '||everyone|| = (λ f_<e,t>: (Forall x_e: f_<e,t>(x_e)))'/<<e,t>,t> and '|||| = (λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: ((λ x_e: Cat_<e,t>(x_e)))(x_e))), (PhysExistence_<e,t>((ι x_e: ((λ x_e: Cat_<e,t>(x_e)))(x_e))) & Sensory_<e,t>(y_e))))'/<e,(t,t)> conflict (mode: Function Application)